# Visualization
## In Rate and Out Rates
We use the historic trip data for one month (May 2024, as the January File format encoding is slightly different and contains NULL end station ids) to calculate the number of bikes docked over a 1-hour window for each minute.

We visualize this data as animated plot, showing a circle at the respective time.

We use `folium` to create the map visualizations and the open Toronto Bikesharing JSON current station data to obtain the capacities. Notably, the capacities of different stations can change over time as stations are made larger, so this would be required to address in the future. We here assume that the number of capacity changes in a year is neglegibly small.

## Goals
We want to analyze demand patterns over the week, month and daytimes.

- Question: Are normal plots also wanted in the Medium article?

Ideas for insightful and visually aesthetic plots:
- Heatmap of minimum-distance-to-border ("MDTB") , i.e. roughly $1 - min(bikes(t), capacity - bikes(t)) / (capacity // 2)$, optionally squared for better visualization of critical areas
- Heatmap of places where the number of intaken bikes is not matched by the out-taken bikes.

## Other Visualizations
- To analyze the connectivity, we can plot the graph obtained by the rides on top of a map.
  



In [31]:
import analysis as an
import importlib
importlib.reload(an)

data = an.BikeShareData.load_from_pickle(name = '2024-5')


In [32]:
data.N_bikes.shape, data.capacities.shape, data.in_rates.shape, len(data.stations)

((994, 83083), (994,), (994, 83083), 994)

In [48]:
import folium
import math

firstkey = data.stations.index[0]
m = folium.Map(location=[data.stations.loc[firstkey, 'lat'], data.stations.loc[firstkey, 'lon']], zoom_start=14)

for _, station in data.stations.iterrows():
    if math.isnan(station['lat']):
        continue
    folium.Marker([station['lat'], station['lon']], popup=station['name'] + str(station['station_id'])).add_to(m)

m


In [13]:
data.stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   station_id  994 non-null    int64  
 1   name        994 non-null    object 
 2   lat         852 non-null    float64
 3   lon         852 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 31.2+ KB


In [42]:
col = plt.cm.viridis(0.5)
type(col)
def argb_to_hex(col):
    return f"#{int(col[3] * 255):02x}{int(col[0] * 255):02x}{int(col[1] * 255):02x}{int(col[2] * 255):02x}"
argb_to_hex(col)

'#ff20908c'

In [46]:
np.max(data.out_bikes)

8

In [44]:
import folium
import math
import numpy as np
import matplotlib.pyplot as plt  # Importing matplotlib.pyplot as plt

# 1. Create a bubble map using folium; each bubble represents the number of bikes at that time point for a station
avg_lat = np.mean(data.stations['lat'])
avg_lon = np.mean(data.stations['lon'])

time = 60 * 12
N_bikes = data.N_bikes[:, time]

m = folium.Map(location=[avg_lat, avg_lon], zoom_start=14)

for _, row in data.stations.iterrows():
    station_id = row['station_id']
    lat = row['lat']
    lon = row['lon']
    name = row['name']
    if math.isnan(lat):
        continue
    
    nbikes = N_bikes[station_id - 7000]
    cap = data.capacities[station_id - 7000]
    true_cap = min(cap, data.stations.loc[station_id - 7000, 'capacity'])
    color = argb_to_hex(plt.cm.viridis(nbikes / true_cap))
    
    folium.Circle(
        [lat, lon], 
        radius=2 * int(nbikes) + 1, 
        popup=name + f" ({nbikes} bikes / {true_cap} capacity)",
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1.0
    ).add_to(m)


m

79 47
717 23
1 17
133 15
22 23
20 31
39 19
46 19
11 25
8 19
13 15
20 19
10 16
200 39
21 35
16 12
64 49
161 55
7 32
30 63
36 39
8 11
974 38
577 23
55 14
9 31
71 31
75 15
1750 35
258 23
3 15
36 43
40 15
5 31
7 19
37 27
34 31
3 19
19 19
11 23
620 31
8 19
268 35
62 37
5 42
70 25
17 47
11 15
257 16
61 55
20 27
10 9
2 15
49 27
17 15
64 15
52 23
6 19
3 15
149 15
10 22
7 11
15 11
124 23
7 23
10 15
10 27
22 57
26 19
11 15
24 19
27 15
3 11
17 15
5 23
4 11
31 15
53 27
4 19
36 15
17 24
2 25
14 15
10 18
4 15
0 0
6 27
6 27
16 30
5 15
2 15
11 15
1 27
2 15
4 15
38 32
3 15
1 11
17 19
8 15
12 27
8 17
1 19
28 19
17 19
2 15
8 10
67 27
1 15
12 15
2 19
6 17
45 15
331 23
8 11
58 15
48 15
26 9
18 11
6 19
28 15
35 27
6 15
36 15
26 15
29 15
6 15
4 15
63 19
4 15
10 23
10 11
10 19
21 19
4 11
28 19
120 23
15 19
15 15
18 23
5 15
3 19
71 23
55 21
24 15
12 23
14 47
4 11
42 15
16 27
6 19
23 19
16 15
7 27
5 19
4 15
85 23
0 15
38 19
3 11
8 11
2 15
9 15
75 15
3 11
0 15
3 15
35 23
12 11
7 10
0 11
5 19
6 19
39 23
12 35
8 1

/tmp/ipykernel_6003/2473903227.py:26: RuntimeWarning: invalid value encountered in scalar divide
  color = argb_to_hex(plt.cm.viridis(nbikes / true_cap))


16 15
4 6
24 17
95 20
7 24
6 9
1 14
2 14
13 15
2 13
0 19
10 14
3 15
5 15
16 19
3 11
2 8
9 18
1 19
10 19
8 12
16 18
20 15
3 24
6 23
7 19
13 11
7 11
6 18
6 15
5 15
0 16
1 15
14 14
6 15
14 15
6 12
9 19
8 16
44 15
20 15
257 19
50 19
1 15
14 14
39 13
26 17
48 23
3 7
23 15
5 11
4 20
5 27
42 19
17 39
2 19
22 17
29 15
3 11
54 26
588 35
411 20
20 12
2 15
2 10
18 15
7 8
15 15
40 19
6 24
2 35
20 23
5 23
4 15
2 11
3 15
17 19
4 26
0 19
6 10
1 19
0 19
22 14
26 15
18 15
15 19
63 18
1 19
5 19
5 19
22 27
4 19
261 27
24 23
366 18
1 15
11 22
27 23
18 9
8 19
1 11
3 11
10 28
30 32
8 11
2 15
43 15
45 19
254 43
167 42
1 15
2 14
10 15
4 8
3 11
0 6
5 7
14 14
3 13
4 9
4 15
28 19
18 18
27 23
2 10
11 13
3 10
0 19
8 15
17 15
5 12
0 23
0 15
13 15
10 15
5 10
6 19
10 13
5 5
0 23
16 15
64 12
21 15
33 14
13 11
23 14
8 13
22 23
0 11
34 14
8 12
3 7
12 11
3 15
3 15
20 11
4 9
21 16
11 15
9 9
11 12
6 16
9 13
3 31
18 23
243 23
1 17
12 15
0 0
12 15
1 19
1 15
14 15
4 8
6 8
1 8
9 10
4 14
5 10
13 15
4 7
14 14
6 8
41 15
7 17
2 5


In [29]:
data.stations

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Wellesley Station Green P,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667131,-79.399555
3,7003,Madison Ave / Bloor St W,43.667018,-79.402796
4,7004,Unknown,NaN,NaN
...,...,...,...,...
989,7989,Don Mills Rd / Fairview Mall Dr,43.778886,-79.348372
990,7990,Don Mills Rd / Goodview Rd,43.785346,-79.353629
991,7991,Kingston Rd / Markham Rd,43.738784,-79.217465
992,7992,Fairglen Cres / Weston Rd,43.708767,-79.534452


In [23]:
from folium.plugins import HeatMap



time = 60 * 12 # roughly 12:00 at 1.05.2024

def mdtb(nbikes, cap):
    return (1 - min(nbikes, cap - nbikes) / (cap // 2)) ** 2

# Define heat data; each row contains lat, lon, (intensity ∈ [0, 1])
heat_data = data.stations[['lat', 'lon']]
heat_data['heat'] = [mdtb(data.N_bikes[i_station, time], cap) for i_station, cap in 
                     enumerate(data.capacities)]
heat_data = heat_data[heat_data['lat'].notna()]
heat_data = heat_data[heat_data['heat'].notna()]
heat_data = heat_data.values.tolist()

avg_lat = sum([x[0] for x in heat_data]) / len(heat_data)
avg_lon = sum([x[1] for x in heat_data]) / len(heat_data)
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

# Add heatmap to the map
HeatMap(heat_data).add_to(m)

# Save the map to an HTML file
m.save('heatmap.html')

/tmp/ipykernel_72048/2744654675.py:8: RuntimeWarning: invalid value encountered in scalar divide
  return (1 - min(nbikes, cap - nbikes) / (cap // 2)) ** 2
/tmp/ipykernel_72048/2744654675.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_data['heat'] = [mdtb(data.N_bikes[i_station, time], cap) for i_station, cap in


[[43.639832, -79.395954, 0.17487603305785127],
 [43.66496415990742, -79.38355031526893, 0.9510279352503244],
 [43.66713121831853, -79.3995550638237, 0.9111570247933886],
 [43.66701830465472, -79.4027958687172, 0.7844897959183673],
 [43.6480008, -79.383177, 0.5048476454293629],
 [43.660439, -79.385525, 0.140625],
 [43.658148, -79.398167, 0.81],
 [43.663376, -79.392125, 0.5917159763313609],
 [43.650325, -79.372287, 0.831744],
 [43.645323, -79.395003, 0.308641975308642],
 [43.656026, -79.385327, 0.2688614540466393],
 [43.64659663170443, -79.37530913867988, 0.0],
 [43.663102, -79.373181, 0.37869822485207105],
 [43.64852, -79.380576, 0.7716207352112415],
 [43.640978, -79.376785, 0.42250000000000004],
 [43.647547634759206, -79.39155200496292, 0.0625],
 [43.641529, -79.386741, 0.8824609733700643],
 [43.650945, -79.379498, 0.0003346222486615116],
 [43.650033, -79.396555, 0.8438345051379124],
 [43.653264, -79.382458, 0.5804988662131518],
 [43.65049, -79.3873, 0.003460207612456749],
 [43.6571, -

In [39]:
import osmnx as ox
ox.plot.get_colors(n=10, cmap='viridis', start=0, stop=1, alpha=1, return_hex=True)

/tmp/ipykernel_6003/1434894996.py:2: FutureWarning: The `return_hex` parameter has been deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.plot.get_colors(n=10, cmap='viridis', start=0, stop=1, alpha=1, return_hex=True)


['#440154ff',
 '#482878ff',
 '#3e4989ff',
 '#31688eff',
 '#26828eff',
 '#1f9e89ff',
 '#35b779ff',
 '#6ece58ff',
 '#b5de2bff',
 '#fde725ff']